In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import time

# Configuración para visualización
#plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("viridis")

# Función para cargar y preparar los datos
def load_data(file_path):
    # Suponemos que tienes los datos en un CSV con el formato mostrado
    # Si tienes los datos en otro formato, adapta esta parte
    df = pd.read_csv(file_path)
    
    # Si no tienes un archivo y quieres usar directamente los datos del ejemplo:
    # Comenta la línea anterior y descomenta las siguientes si necesitas crear un DataFrame desde los datos proporcionados
    '''
    # Datos de ejemplo (adapta según los datos completos)
    data = [
        [25521,-1.2148779857976169,-0.6009048715002394,1,2,1947,-8.17,43.2,24.75,"Apartamento","Media","Azul",-0.08644017968125128,-1.0638780790348752,0.10216600321123659,"Bo","Norte","C",0,51938,78,-0.21722361560963993,-0.6886523106404716,0.46976204453079845,1.270152276635409,-0.4122399603482988,-1.6095783677270268,-0.12641618406509117,-1.0683974353679875,1.0499949614135753,2.0,1,5,2,3,1,0,0,0,1,0,0,0,0],
        # Añade más filas según sea necesario
    ]
    columns = ["id","superficie_interior_m2","superficie_exterior_m2","numero_habitacions","numero_banos","ano_construccion","lonxitude","latitude","temperatura_media_mes_construccion","tipo_edificacion","calidade_materiais","cor_favorita_propietario","distancia_centro_km","distancia_escola_km","indice_criminalidade","acceso_transporte_publico","orientacion","eficiencia_enerxetica","numero_arboles_xardin","prezo_euros","edad_vivienda","superficie_por_habitacion","superficie_total","ratio_interior_exterior","densidad_banos","densidad_habitaciones","dist_coruna","dist_vigo","dist_santiago","calidad_edad","banos_por_habitacion","orientacion_valor","eficiencia_valor","calidade_valor","transporte_valor","tipo_Apartamento","tipo_Casa","tipo_Chalet adosado","color_Amarelo","color_Azul","color_Branco","color_Negro","color_Verde","color_Vermello"]
    df = pd.DataFrame(data, columns=columns)
    '''
    
    # Verificar y gestionar valores faltantes
    print(f"Valores faltantes en el dataset: {df.isnull().sum().sum()}")
    
    # Separamos features y target
    X = df.drop(['prezo_euros', 'id'], axis=1, errors='ignore')
    y = df['prezo_euros']
    
    print(f"Forma del dataset: {df.shape}")
    print(f"Features incluidas: {X.columns.tolist()}")
    
    return X, y

# Función para entrenar y evaluar modelos
def train_and_evaluate_models(X, y, random_state=42):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state
    )
    
    # Configuración para validación cruzada
    kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    # Modelos a comparar
    models = {
        'XGBoost': XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            min_child_weight=1,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='reg:squarederror',
            n_jobs=-1,
            random_state=random_state
        ),
        'CatBoost': CatBoostRegressor(
            iterations=500,
            learning_rate=0.05,
            depth=6,
            loss_function='MAE',
            verbose=100,  # Reducir para menos output
            random_state=random_state
        )
    }
    
    results = {}
    importances = {}
    trained_models = {}
    
    # Entrenar y evaluar cada modelo
    for name, model in models.items():
        start_time = time.time()
        print(f"\n{'='*50}\nEntrenando modelo: {name}\n{'='*50}")
        
        # Entrenamiento con validación cruzada
        cv_scores = cross_val_score(
            model, X_train, y_train, 
            cv=kfold, 
            scoring='neg_mean_absolute_error', 
            n_jobs=-1
        )
        
        # Entrenar modelo completo
        model.fit(X_train, y_train)
        trained_models[name] = model
        
        # Predecir en conjunto de prueba
        y_pred = model.predict(X_test)
        
        # Cálculo de métricas
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # Tiempo de entrenamiento
        training_time = time.time() - start_time
        
        # Almacenar resultados
        results[name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'CV_MAE': -np.mean(cv_scores),
            'CV_MAE_std': np.std(cv_scores),
            'Training_Time': training_time
        }
        
        # Almacenar importancias de características
        if name == 'XGBoost':
            importances[name] = model.feature_importances_
        elif name == 'CatBoost':
            importances[name] = model.get_feature_importance()
        
        print(f"Métricas del modelo {name}:")
        print(f"  MAE: {mae:.2f}")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  R2: {r2:.4f}")
        print(f"  CV MAE: {-np.mean(cv_scores):.2f} (±{np.std(cv_scores):.2f})")
        print(f"  Tiempo de entrenamiento: {training_time:.2f} segundos")
    
    return results, importances, trained_models, X_test, y_test

# Función para visualizar resultados
def visualize_results(results, importances, X, trained_models, X_test, y_test):
    # 1. Comparación de métricas
    metrics_df = pd.DataFrame({
        'Modelo': list(results.keys()),
        'MAE': [results[model]['MAE'] for model in results],
        'RMSE': [results[model]['RMSE'] for model in results],
        'R²': [results[model]['R2'] for model in results],
        'CV MAE': [results[model]['CV_MAE'] for model in results],
        'Tiempo (s)': [results[model]['Training_Time'] for model in results]
    })
    
    print("\n=== Comparación de modelos ===")
    print(metrics_df.to_string(index=False))
    
    # 2. Visualización de MAE
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Modelo', y='MAE', data=metrics_df)
    plt.title('Comparación de Error Absoluto Medio (MAE)', fontsize=15)
    plt.ylabel('MAE (¬)', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('mae_comparison.png')
    plt.close()
    
    # 3. Top características más importantes para cada modelo
    for model_name, importance in importances.items():
        # Crear DataFrame con importancias
        imp_df = pd.DataFrame({
            'Feature': X.columns,
            'Importance': importance
        }).sort_values(by='Importance', ascending=False)
        
        # Mostrar top 15 características
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=imp_df.head(15))
        plt.title(f'Top 15 características más importantes - {model_name}', fontsize=15)
        plt.tight_layout()
        plt.savefig(f'{model_name}_feature_importance.png')
        plt.close()
    
    # 4. Predicciones vs valores reales
    plt.figure(figsize=(14, 7))
    
    for i, (name, model) in enumerate(trained_models.items()):
        y_pred = model.predict(X_test)
        
        plt.subplot(1, 2, i+1)
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.title(f'{name}: Predicciones vs Reales', fontsize=15)
        plt.xlabel('Precio Real (¬)', fontsize=12)
        plt.ylabel('Precio Predicho (¬)', fontsize=12)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('predictions_vs_actual.png')
    plt.close()
    
    # 5. Distribución de errores
    plt.figure(figsize=(14, 7))
    
    for i, (name, model) in enumerate(trained_models.items()):
        y_pred = model.predict(X_test)
        errors = y_test - y_pred
        
        plt.subplot(1, 2, i+1)
        sns.histplot(errors, kde=True, bins=30)
        plt.axvline(x=0, color='r', linestyle='--')
        plt.title(f'{name}: Distribución de errores', fontsize=15)
        plt.xlabel('Error (¬)', fontsize=12)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('error_distribution.png')
    plt.close()
    
    # 6. Determinar el mejor modelo
    best_model = min(results.items(), key=lambda x: x[1]['MAE'])[0]
    print(f"\n El mejor modelo según MAE es: {best_model}")
    print(f"   MAE: {results[best_model]['MAE']:.2f}")
    print(f"   RMSE: {results[best_model]['RMSE']:.2f}")
    print(f"   R²: {results[best_model]['R2']:.4f}")

# Función principal
def main():
    print("Iniciando análisis comparativo de modelos para predicción de precios inmobiliarios...")
    
    # Cargar datos - ajusta la ruta al archivo CSV
    X, y = load_data('train_processed.csv')
    
    # Entrenar y evaluar modelos
    results, importances, trained_models, X_test, y_test = train_and_evaluate_models(X, y)
    
    # Visualizar resultados
    visualize_results(results, importances, X, trained_models, X_test, y_test)
    
    print("\nAnálisis completado. Se han generado gráficos de comparación.")

if __name__ == "__main__":
    main()

Iniciando análisis comparativo de modelos para predicción de precios inmobiliarios...
Valores faltantes en el dataset: 0
Forma del dataset: (20000, 47)
Features incluidas: ['superficie_interior_m2', 'superficie_exterior_m2', 'numero_habitacions', 'numero_banos', 'ano_construccion', 'lonxitude', 'latitude', 'temperatura_media_mes_construccion', 'distancia_centro_km', 'distancia_escola_km', 'indice_criminalidade', 'numero_arboles_xardin', 'edad_vivienda', 'superficie_por_habitacion', 'superficie_total', 'ratio_interior_exterior', 'densidad_banos', 'densidad_habitaciones', 'dist_coruna', 'dist_vigo', 'dist_santiago', 'calidad_edad', 'banos_por_habitacion', 'orientacion_valor', 'eficiencia_valor', 'calidade_valor', 'transporte_valor', 'tipo_Apartamento', 'tipo_Casa', 'tipo_Chalet adosado', 'color_Amarelo', 'color_Azul', 'color_Branco', 'color_Negro', 'color_Verde', 'color_Vermello', 'tipo_Apartamento.1', 'tipo_Casa.1', 'tipo_Chalet adosado.1', 'color_Amarelo.1', 'color_Azul.1', 'color_Bran